## generate screenshot data

In [1]:
from cairosvg import svg2png
import chess
import chess.pgn
import chess.svg
import json
from random import randint
from pathlib import Path
from IPython import display
from PIL import Image, ImageDraw
from dataclasses import dataclass
from typing import List, Dict, Tuple
from pdb import set_trace
from itertools import zip_longest
import pandas as pd
from io import BytesIO

In [2]:
class BoardGenerator:
    def __init__(self, pgn):
        self.pgn = open(pgn)
    
    def get_boards(self):
        boards = []
        for i in range(30):
            game = chess.pgn.read_game(self.pgn)
            board = game.board()
            for i, move in enumerate(game.mainline_moves()):
                board.push(move)
                boards.append(board.copy())

        # todo: use yield
        return boards

In [15]:
pgn = Path("../data/pgn/kasparov_1798.pgn")
bg = BoardGenerator(pgn)
boards = bg.get_boards()

In [16]:
class SSComposite:
    """ class for compositing a board onto a screenshot"""
    
    def __init__(self, df, output_dir):
        self.df = df
        self.output_dir = output_dir
        self.anno = {}
        self.anno['train'] = []
        self.anno['test'] = []
        self.anno['valid'] = []

    def composite(self):
        """ composite boards with screenshot"""
        
        for _, row in df.iterrows():
            image_f = row['image_f']
            dataset = row['dataset']
            board = row['board']
            
            # BUG IS HERE
            min_size = 100
            image = Image.open(image_f.as_posix())

            x = randint(0, image.size[0] - min_size)
            y = randint(0, image.size[1] - min_size)
            max_size = min(image.size[0] - x, image.size[1] - y)
            board_size = randint(min_size, max_size)
            board_image =  Image.open(BytesIO(self._render_board(board, board_size)))
            image.paste(board_image, (x, y))

            # adjust x y so it's at center of board as excepted by createml
            coord = {'x': x + int(board_size/2),
                 'y': y + int(board_size/2),
                 'width': board_size,
                 'height': board_size}
            
            image.save(f"{self.output_dir}/{dataset}/{image_f.name}")
            self.anno[dataset].append({
                'image': image_f.name, 
                'annotations': [{'label': 'chessboard', "coordinates": coord}]
            })
            
    def save_annotations(self):
        for o in ['train', 'test', 'valid']:
            with open(f'{self.output_dir}/{o}/{o}.json', 'w') as json_file:
                json.dump(self.anno[o], json_file)
    
    def _render_board(self, board, size):
        s = chess.svg.board(board,
                            size=size,
                            coordinates=randint(0, 1),
                            orientation=randint(0, 1))
        return svg2png(bytestring = s)

In [17]:
data = Path("/home/idris/.fastai/data/websitescreenshots")
data_out = Path("/home/idris/.fastai/data/chess_screenshots")

In [18]:
createml = Path(data/"test/_annotations.createml.json")

In [19]:
def get_images(folder):
    return [_ for _ in data.glob(f"{folder}/*.jpg") if not _.name.startswith('._')]

train_imgs = get_images('train')
test_imgs = get_images('test')
val_imgs = get_images('valid')

In [20]:
a = list(zip_longest(train_imgs, [], fillvalue='train'))
b = list(zip_longest(test_imgs, [], fillvalue='test'))
c = list(zip_longest(val_imgs, [], fillvalue='valid'))
a.extend(b)
a.extend(c)
comps = zip([_[0] for _ in a], [_[1] for _ in a], boards)

In [21]:
df = pd.DataFrame(list(comps), columns=['image_f', 'dataset', 'board'])

In [22]:
ss = SSComposite(df[:10], data_out)
ss.composite()

In [23]:
ss.save_annotations()

In [12]:
df[:10]

,image_f,dataset,board
0,/home/idris/.fastai/data/websitescreenshots/tr...,train,r n b q k b n r\np p p p p p p p\n. . . . . . ...
1,/home/idris/.fastai/data/websitescreenshots/tr...,train,r n b q k b n r\np p p p . p p p\n. . . . p . ...
2,/home/idris/.fastai/data/websitescreenshots/tr...,train,r n b q k b n r\np p p p . p p p\n. . . . p . ...
3,/home/idris/.fastai/data/websitescreenshots/tr...,train,r n b q k b n r\np p p . . p p p\n. . . . p . ...
4,/home/idris/.fastai/data/websitescreenshots/tr...,train,r n b q k b n r\np p p . . p p p\n. . . . p . ...
5,/home/idris/.fastai/data/websitescreenshots/tr...,train,r . b q k b n r\np p p . . p p p\n. . n . p . ...
6,/home/idris/.fastai/data/websitescreenshots/tr...,train,r . b q k b n r\np p p . . p p p\n. . n . p . ...
7,/home/idris/.fastai/data/websitescreenshots/tr...,train,r . b q k b . r\np p p . . p p p\n. . n . p . ...
8,/home/idris/.fastai/data/websitescreenshots/tr...,train,r . b q k b . r\np p p . . p p p\n. . n . p . ...
9,/home/idris/.fastai/data/websitescreenshots/tr...,train,r . b q k b . r\np p p . . . p p\n. . n . p p ...
